# RFP: Maze Solvers

## Project Overview
You are invited to submit a proposal that answers the following question:

### What path will your elf take?

*Please submit your proposal by **2/11/25 at 11:59 PM**.*

## Required Proposal Components

### 1. Data Description
In the code cell below, use [Gymnasium](https://gymnasium.farama.org/) to set up a [Frozen Lake maze](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) for your project. When you are done with the set up, describe the reward system you plan on using.

*Note, a level 5 maze is at least 10 x 10 cells large and contains at least five lake cells.*

In [1]:
import gymnasium as gym
from gymnasium.envs.toy_text.frozen_lake import generate_random_map
import random
import numpy as np

In [2]:
# Make maze

env = gym.make('FrozenLake-v1', desc=generate_random_map(size=10, seed=259),  is_slippery=False)

In [3]:
# Get state and action sizes
state_size = env.observation_space.n
action_size = env.action_space.n

# Extract hole ('H'), empty ('F'), and goal ('G') positions
desc = env.unwrapped.desc
hole_states = {i for i, row in enumerate(desc.flatten()) if row == b'H'}
empty_states = {i for i, row in enumerate(desc.flatten()) if row == b'F'}
goal_state = {i for i, row in enumerate(desc.flatten()) if row == b'G'}

# Initialize Q-table with small random values to encourage exploration
qtable = np.random.uniform(low=-0.5, high=0.5, size=(state_size, action_size))

#### Describe your reward system here.

### 2. Training Your Model
In the cell seen below, write the code you need to train a Q-Learning model. Display your final Q-table once you are done training your model.

*Note, level 5 work uses only the standard Python library and Pandas to train your Q-Learning model. A level 4 uses external libraries like Baseline3.*

In [4]:
# Training parameters
total_episodes = 10000  # Increase episodes for better learning
max_steps = 200  
learning_rate = 0.95  # Keep high at the start for faster learning
gamma = 0.75  # Higher discount factor to encourage long-term rewards
epsilon = 0.9  
min_epsilon = 0.01  # Allow more exploration for longer
decay_rate = 0.005  # Slower decay to ensure sufficient exploration

In [5]:
# Training loop
rewards = []
for episode in range(total_episodes):
    state, _ = env.reset()
    total_rewards = 0
    done = False

    for step in range(max_steps):
        # Choose action using epsilon-greedy strategy
        if random.uniform(0, 1) > epsilon:
            action = np.argmax(qtable[state])  # Exploit
        else:
            action = env.action_space.sample()  # Explore

        # Take action
        new_state, reward, done, truncated, _ = env.step(action)

        # Modify rewards
        if new_state in hole_states:
            reward = -1.0  # Higher penalty for falling in a hole
        elif new_state in empty_states:
            reward = 0.1  # Small reward for moving forward
        elif new_state in goal_state:
            reward = 1.0  # Higher reward for reaching goal

        # Q-learning update
        qtable[state, action] = qtable[state, action] + learning_rate * (
            reward + gamma * np.max(qtable[new_state]) - qtable[state, action]
        )

        total_rewards += reward
        state = new_state

        if done or truncated:
            break

    # Decay epsilon slower for better exploration
    epsilon = max(min_epsilon, epsilon * np.exp(-decay_rate * episode))
    rewards.append(total_rewards)

### 3. Testing Your Model
In the cell seen below, write the code you need to test your Q-Learning model for **1000 episodes**. It is important to test your model for 1000 episodes so that we are all able to compare our results.

*Note, level 5 testing uses both a success rate and an average steps taken metric to evaluate your model. Level 4 uses one or the other.*

In [6]:
# Print results
print("Score over time:", sum(rewards) / total_episodes)
print("Q-Table:")
print(qtable)

Score over time: 9.953549999999796
Q-Table:
[[ 3.00000000e-01  4.00000000e-01  4.00000000e-01  3.00000000e-01]
 [ 3.00031901e-01  4.00000000e-01  4.00000000e-01  4.00000000e-01]
 [ 4.00000000e-01  4.00000000e-01  4.00000000e-01  4.00000000e-01]
 [ 4.00000000e-01 -6.73063628e-01  3.99957196e-01  4.00000000e-01]
 [ 4.00000000e-01  2.78134477e-01 -1.05292076e+00  3.81312911e-01]
 [-4.27523192e-01 -4.24626519e-01 -1.62042296e-01 -2.31281877e-01]
 [-1.08155942e+00  4.00000000e-01  3.73288719e-01  3.97854330e-01]
 [-2.83429152e-01  3.98475884e-01 -3.35347739e-01 -7.21790912e-02]
 [ 2.68629644e-01 -1.32201603e-01  4.51311623e-01 -1.68233624e-01]
 [-4.60403941e-01  4.18460930e-01 -1.35747892e-01  4.36371074e-01]
 [ 4.00000000e-01  4.00000000e-01  4.00000000e-01  3.00000000e-01]
 [ 4.00000000e-01  4.00000000e-01  4.00000000e-01  4.00000000e-01]
 [ 4.00000000e-01  4.00000000e-01 -6.73063628e-01  4.00000000e-01]
 [-4.72646898e-01 -2.26921604e-01  4.35915163e-01 -2.55006833e-01]
 [-1.02366911e-01 

In [ ]:
# Variables to track success and steps
total_episodes = 1000  # Number of episodes to run
successful_episodes = 0
total_steps = 0

for episode in range(total_episodes):
    state = env.reset()[0]  # Reset environment and get initial state
    done = False
    steps = 0

    while not done:
        action = np.argmax(qtable[state, :])  # Pick the best action based on Q-values
        new_state, reward, done, truncated, _ = env.step(action)  # Take action
        state = new_state  # Move to the next state
        steps += 1

    total_steps += steps

    if reward == 1:
        successful_episodes += 1

env.close()  # Close the environment

# Calculate success rate and average steps per episode
success_rate = successful_episodes / total_episodes
average_steps = total_steps / total_episodes

print(f"Success Rate: {success_rate * 100:.2f}%")
print(f"Average Steps per Episode: {average_steps:.2f}")

### 4. Final Answer
In the first cell below, describe the path your elf takes to get to the gift. *Note, a level 5 answer includes a gif of the path your elf takes in order to reach the gift.*

In the second cell seen below, describe how well your Q-Learning model performed. Make sure that you explicitly name the **learning rate**, **the discount factor**, and the **reward system** that you used when training your final model. *Note, a level 5 description describes the model's performance using two types of quantitative evidence.*

![example image](https://gymnasium.farama.org/_images/frozen_lake.gif)

#### Describe the path your elf takes here.


![My GIF](ezgif.com-video-to-gif-converter.gif)

#### Describe how well your Q-Learning model performed here.

##### My Q-Learning Model performed very well as to get the elf to the gift, with only 18 steps. The learning rate I put was a good learning rate because the elf looks like he has learned from learning all over the frozen lakes and is given a great learning model with the discount rate being higher so that the elf can get long term rewards. My reward system consists of a 0.001 reward for an empty space, a -1.1 reward for falling into a hole, and a +1 reward for reaching the gift. As you can see from the gif this is the path it consistantly goes now, and the path really secured a solid path to the gift. Other things that I could consider is that my elf could be put on a slippery frozen lake and still perform just as well. If you have taken a look at my code, I used a strategy to epsilon, by flipping a coin on whether it should exploit or explore. Another thing I did add was a decrease in decay rate so that the elf could explore more than exploit what it knows.